# **Face Emotion Detection and Moments Capture**

This project captures Real-time inputs from the user, takes in account of the harrcascade_frontalface_default for Face Detection, predicts the Emotion of the person using Pretrained Model (XceptionNet) and saves the pictures of the moods that are marked for it to save if has a particular Confidence Threshold

### **Imports**

In [1]:
import os
import cv2
import time
import numpy as np
from keras.models import load_model

### **Loading the Pretrained Emotion Detection Model**

In [2]:
model = load_model('emotion_model.hdf5',compile=False)
try:
    os.mkdir("./selfies")
except FileExistsError:
    print("Folder exists!")

Folder exists!


### **Utility Functions**

In [36]:
def preprocess(image):
    image = cv2.resize(image, (64, 64))
    image = image.astype('float32') / 255
    image = np.expand_dims(image, 0)
    image = np.expand_dims(image, -1)
    return image

def draw_bounding_box(face_coordinates, image_array, color):
    x, y, w, h = face_coordinates
    cv2.rectangle(image_array, (x, y), (x + w, y + h), color, 3)

def apply_offsets(face_coordinates, offsets):
    x, y, width, height = face_coordinates
    x_off, y_off = offsets
    return (x - x_off, x + width + x_off, y - y_off, y + height + y_off)
    # return (x, x + width, y, y + height)

def draw_text(coordinates, image_array, text, color, x_offset=0, y_offset=0, font_scale=2, thickness=2):
    x, y = coordinates[:2]
    cv2.putText(image_array, text, (x + x_offset, y + y_offset),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale, color, thickness, cv2.LINE_AA)

### **Hyperparameters**

In [7]:
SELFIE_PROBABILITY = 0.8
selfie_emotions = ["happy", "disgust", "sad"]


colors = {
    'happy': np.asarray((0, 255, 0)),
    'angry': np.asarray((255, 0, 0)),
    'sad': np.asarray((0, 0, 255)),
    'surprise': np.asarray((255, 255, 0))
}

COLOR_ON_PROBABILITY = False

emotion_model_path = 'emotion_model.hdf5'
emotion_labels = {
    0:'angry',
    1:'disgust',
    2:'fear',
    3:'happy',
    4:'sad',
    5:'surprise',
    6:'neutral'
}

face_cascade = cv2.CascadeClassifier('harrcascade_frontalface_default.xml')
emotion_classifier = load_model(emotion_model_path)

emotion_offsets = (20, 40)
emotion_target_size = emotion_classifier.input_shape[1:3]

### **Realtime Detection and Moments Capture**

In [42]:
video = cv2.VideoCapture(0)

while True:
    success, frame = video.read()

    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=4)

    for face_coordinates in faces:
        x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
        # cv2.rectangle(frame, (x1,y1),(x2,y2),color=(0,255,0),thickness=3)
        gray_face = gray_image[y1:y2, x1:x2]
        try:
            gray_face = cv2.resize(gray_face, (emotion_target_size))
        except:
            continue

        gray_face = preprocess(gray_face)
        emotion_prediction = emotion_classifier.predict(gray_face)
        emotion_probability = np.max(emotion_prediction)
        emotion_label_arg = np.argmax(emotion_prediction)
        emotion_text = emotion_labels[emotion_label_arg]
        
        print_text = f"{emotion_text} {str(emotion_probability.round(2))}" 
        print(f"\r{print_text}")

        if (emotion_text in selfie_emotions) and (emotion_probability > SELFIE_PROBABILITY):
            cv2.imwrite(f"./selfies/{emotion_text}_{time.time()}.jpg",frame)
            print("Selfie Captured!")

        try:
            color = colors[emotion_text]
        except KeyError:
            print(f"No color preset for {emotion_text}")
            color = np.asarray((0, 255, 255))

        color *= emotion_probability if COLOR_ON_PROBABILITY else 1
        color = color.astype(int)
        color = color.tolist()

        draw_bounding_box(face_coordinates, rgb_image, color)
        draw_text(face_coordinates, rgb_image, print_text,
                  color, -15, -20, 0.75, 2)

    frame = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
    cv2.imshow('Emotion Analyser', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

video.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 23ms/step
neutral 0.41
No color preset for neutral
1/1 [==============================] - 0s 27ms/step
angry 0.49
1/1 [==============================] - 0s 27ms/step
angry 0.4
1/1 [==============================] - 0s 22ms/step
neutral 0.59
No color preset for neutral
1/1 [==============================] - 0s 22ms/step
angry 0.38
1/1 [==============================] - 0s 26ms/step
happy 0.63
1/1 [==============================] - 0s 22ms/step
neutral 0.39
No color preset for neutral
1/1 [==============================] - 0s 23ms/step
angry 0.42
1/1 [==============================] - 0s 22ms/step
neutral 0.47
No color preset for neutral
1/1 [==============================] - 0s 25ms/step
angry 0.41
1/1 [==============================] - 0s 22ms/step
neutral 0.48
No color preset for neutral
1/1 [==============================] - 0s 24ms/step
angry 0.44
1/1 [==============================] - 0s 22ms/step
neutral 0.5
No color preset for neutral
1/1